<a href="https://colab.research.google.com/github/mille-s/GEM24_EvalLLM/blob/main/GEM24_EvalLLM_aiXplain_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Aixplain
from IPython.display import clear_output
! pip install aixplain
clear_output()

In [ ]:
#@title Download and Load human-eval-packaged json, and format contents (triples, text, id)
import json
import codecs
from bs4 import BeautifulSoup
import os


language = "Spanish" #@param["English", "Spanish", "Swahili"]
data = 'regular' #@param["regular", "iaa"]

def format_json(json_path):
  # Open en_regular and parse json
  en_regular_json = json.load(codecs.open(json_path, 'r', 'utf-8'))
  # Print first entry
  # print(json.dumps(en_regular_json[0], indent=4))

  triples_text_pairs = []

  x = 0
  while x < len(en_regular_json):
    # if x < 10:
    # Parse html found in the "input" key
    html = en_regular_json[x]['input']
    soup = BeautifulSoup(html, 'html.parser')
    # Print raw table
    # print(soup.prettify())
    table = soup.find('table')
    # headers = [header.text.strip() for header in table.find_all('th')]
    rows = []
    for row in table.find_all('tr'):
      columns = row.find_all(['td', 'th'])  # Get both <td> and <th>
      row_data = ' '.join([col.text.strip() for col in columns])
      rows.append(row_data)
    triples_formatted = '; '.join(rows[1:]) # exclude header
    # print("Headers:", rows[0])
    # print(rows[1:])
    triples_text_pairs.append({'id':en_regular_json[x]['id'], 'triples': '"""'+triples_formatted+'"""', 'text': en_regular_json[x]['output']})
    # else:
    #   break
    x += 1
  return triples_text_pairs

if language == "English":
  if data == 'regular':
    if not os.path.exists('en_regular.json'):
      ! gdown 1hOp1_zN2IgGeTnmzngOgIdLRXErchkEo
    triples_text_pairs = format_json('en_regular.json')
  elif data == 'iaa':
    if not os.path.exists('en_iaa.json'):
      ! gdown 1Q69osoEzJJBVqYH9t2ArPAaPAHVPNax5
    triples_text_pairs = format_json('en_iaa.json')

elif language == "Spanish":
  if data == 'regular':
    if not os.path.exists('es_regular.json'):
      ! gdown 14hNTAfMqQI2-K81O9GZ4GkoybI2iO-sh
    triples_text_pairs = format_json('es_regular.json')
  elif data == 'iaa':
    if not os.path.exists('es_iaa.json'):
      ! gdown 1HadgIdWzcb7NTJYELKB-MJ-qa510e3M4
    triples_text_pairs = format_json('es_iaa.json')

elif language == "Swahili":
  if data == 'regular':
    if not os.path.exists('sw_regular.json'):
      ! gdown 1Knp231CEdUQ-SDnJ_oknJDm2dogetSFg
    triples_text_pairs = format_json('sw_regular.json')
  elif data == 'iaa':
    if not os.path.exists('sw_iaa.json'):
      ! gdown 16JFZg8DacuhRvIAUJ0iMVqnvh4nrxZTX
    triples_text_pairs = format_json('sw_iaa.json')

In [ ]:
#@title Print sample datapoint
expected = 0
if language == "English":
  if data == 'regular':
    expected = 8280
    print('Expected: 8280')
  elif data == 'iaa':
    expected = 92
    print('Expected: 92')
elif language == "Spanish":
  if data == 'regular':
    expected = 3240
    print('Expected: 3240')
  elif data == 'iaa':
    expected = 72
    print('Expected: 72')
elif language == "Swahili":
  if data == 'regular':
    expected = 2700
    print('Expected: 2700')
  elif data == 'iaa':
    expected = 60
    print('Expected: 60')
print(f'Found: {len(triples_text_pairs)}')

if not len(triples_text_pairs) == expected:
  print('--------------------------------------------------\n!!! ERROR: mismatch expected/found data points !!!\n--------------------------------------------------')

print(triples_text_pairs[0])

# Triples = '"""Marcus_Aurelius HasChild Fadilla; Marcus_Aurelius StudentOf Alexander_of_Cotiaeum; Marcus_Aurelius Spouse Faustina_the_Younger; Marcus_Aurelius PositionHeld Roman_emperor"""'
# Nice_Text = '''Marcus Aurelius was a student of Alexander of Cotiaeum and had a child named Fadilla. He was married to Faustina the Younger. He was a Roman emperor and died in Vindobona.'''

In [ ]:
#@title Run evaluation (needs aiXplain API key in Parameters)
import os
import json
import csv
import pickle

# PARAMETERS aiXplain
#==========================
os.environ["AIXPLAIN_API_KEY"] = 'INSERT_KEY_HERE'
Gemini_1dot5_Flash = '674b73f06eb563a748561d41' # Code double checked
path_out = 'Gemini_results'
#==========================
from aixplain.factories import AgentFactory
# from aixplain.modules.agent import ModelTool
from aixplain.modules.agent.tool.model_tool import ModelTool

if not os.path.exists(path_out):
  os.makedirs(path_out)

def dumpResults(annotations, path_out):
  results_file = open(os.path.join(path_out, 'All_Gemini_results'), 'ab')
  pickle.dump(annotations, results_file)
  results_file.close()

def callGemini_aiXplain(prompt, model):
  agent = AgentFactory.create(
    name="Assessment of text quality",
	  description="Assessment of text quality",
	  instructions="",
    tools=[
      ModelTool(model=model),
    ],
  )
  agent_response = agent.run(prompt)

  return agent_response

def runEval(triples_text_pairs, model):
  # EN regular splits: range(0, 2750), range(2750, 5500), range(5500, 8280)
  x = 0
  # To get all evaluations
  for x in range(0, len(triples_text_pairs)):
  # for x in range(7413, 8280):
  # while x < 3:
    Triples = triples_text_pairs[x]['triples']
    Nice_Text = triples_text_pairs[x]['text']
    id = triples_text_pairs[x]['id']

    #Prompt (Do not change unless discussed with the GEM-HumEval group)
    prompt = '''
In this task, you will evaluate the quality of the Text in relation to the given Triple Set. How well does the Text represent the Triple Set?  You will be given four specific Dimensions to evaluate against:

Dimensions:"""
No-Omissions: ALL the information in the Triple Set is present in the Text.
No-Additions: ONLY information from the Triple Set is present in the Text.
Grammaticality: The Text is free of grammatical and spelling errors.
Fluency: The Text flows well and is easy to read; its parts are connected in a natural way."""

Important note on No-Omissions and No-Additions: some Triple Set/Text pairs contain non-factual information and even fictional names for people, places, dates, etc. Whether there are omissions and/or additions in a Text is NOT related to factual truth, but instead is strictly related to the contents of the input Triple Set.
Important note on Grammaticality and Fluency: for Grammaticality and Fluency you do not need to consider the input Triple Set; only the intrinsic quality of the Text needs to be assessed.

You need to provide the scores ranging from 1 (indicating the lowest score) to 7 (indicating the highest score) for each of the dimensions and a short justification for each score in the following JSON format:  {"No-Omissions": {"Justification": "", "Score": ""}, "No-Additions": {"Justification": "", "Score": ""}, "Grammaticality": {"Justification": "", "Score": ""}, "Fluency": {"Justification": "", "Score": ""} }.

Make sure to read thoroughly the Triple Set and the '''+str(language)+''' Text below, and assess the four Dimensions using the instructions and template above.

Triple Set: ''' + str(Triples) + "\n" + '''Text: '''+ str(Nice_Text) + "\n\n" + '''
'''
    print(f'Evaluating text #{x}...')

    # To run missing evals
    # if id == 'en_D2T-1-FA_1318_6' or id == 'en_D2T-1-FA_0342_6':
    # if id == 'es_D2T-1-FA_0614_2':
    # Begin indent block
    responseGemini = callGemini_aiXplain(prompt, model)
    # Gemini does not always output correct json format, so I save the raw result at this point
    triples_text_pairs[x]['scores_Gemini'] = str(responseGemini['data']['output'])

    # Save individual files as backup
    with open(os.path.join('Gemini_results', 'Gemini_results_'+str(id)), 'ab') as f:
      pickle.dump(triples_text_pairs[x], f)
    # End indent block

    x += 1

  return triples_text_pairs

annotations = runEval(triples_text_pairs, Gemini_1dot5_Flash)

# dumpResults(annotations, path_out)

In [ ]:
#@title Zip and download Gemini_results folder
from google.colab import files
import datetime

date = datetime.date.today().strftime("%y%m%d")
zip_name = f'{date}_{language}_GeminiFlash1dot5_{data}_{str(len(triples_text_pairs))}.zip'
# zip_name = f'{date}_{language}_{model}_{data}_0001-2750.zip'

!zip -r /content/{zip_name} /content/Gemini_results
files.download(zip_name)

## Results analysis

In [ ]:
#@title Unzip uploaded file
import os
import zipfile
from IPython.display import clear_output

# !unzip /content/GPT_results.zip -d /content
zip_language = 'ES' #@param['EN', 'ES', 'SW']
zip_model = 'Gemini-1dot5-flash' #@param['GPT-4o-mini', 'GPT-o3-mini', 'Gemini-1dot5-flash']
zip_data = 'regular' #@param['regular', 'iaa']

def unzipEvals(path_dir, zip_path):
  if os.path.exists(path_dir):
    # Remove folder contents
    for filename in os.listdir(path_dir):
      file_path = os.path.join(path_dir, filename)
  else:
    os.makedirs(path_dir)
  ! unzip {zip_path} -d {path_dir}

path_dir = os.path.join(zip_language+'_'+zip_data, zip_model)
zip_path = ''
if zip_data == 'regular':
  if zip_model == 'GPT-4o-mini':
    if zip_language == 'EN':
      zip_path = '/content/250207_English_gpt-4o-mini-2024-07-18_regular_8280.zip'
    elif zip_language == 'ES':
      zip_path = '/content/250211_Spanish_gpt-4o-mini-2024-07-18_regular_3240.zip'
    elif zip_language == 'SW':
      zip_path = '/content/250211_Swahili_gpt-4o-mini-2024-07-18_regular_2700.zip'
  elif zip_model == 'GPT-o3-mini':
    if zip_language == 'EN':
      zip_path = '/content/250213_English_o3-mini-2025-01-31_regular_8280.zip'
    elif zip_language == 'ES':
      zip_path = '/content/250212_Spanish_o3-mini-2025-01-31_regular_3240.zip'
    elif zip_language == 'SW':
      zip_path = '/content/250214_Swahili_o3-mini-2025-01-31_regular_2700.zip'
  elif zip_model == 'Gemini-1dot5-flash':
    if zip_language == 'EN':
      zip_path = '/content/250227_English_GeminiFlash1dot5_regular_8280.zip'
    elif zip_language == 'ES':
      zip_path = '/content/250225_Spanish_GeminiFlash1dot5_regular_3240.zip'
    elif zip_language == 'SW':
     zip_path = '/content/250225_Swahili_GeminiFlash1dot5_regular_2700.zip'

unzipEvals(path_dir, zip_path)

clear_output()

In [ ]:
#@title Load unzipped files
import pickle
import glob
import json
import re
import codecs

update_params_unzip = False #@param {type:"boolean"}
if update_params_unzip:
  zip_language = 'ES' #@param['EN', 'ES', 'SW']
  zip_model = 'Gemini-1dot5-flash' #@param['GPT-4o-mini', 'GPT-o3-mini', 'Gemini-1dot5-flash']
  zip_data = 'regular' #@param['regular', 'iaa']

model_scores = [1, 2, 3, 4, 5, 6, 7]
# load_gemini_folder = True #@param {type:"boolean"}
# load_gpt_folder = False #@param {type:"boolean"}
path_dir_unzipped = ''
model_prefix = ''
if zip_model.startswith('Gemini'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, 'content', 'Gemini_results')
  model_prefix = 'Gemini'
elif zip_model.startswith('GPT'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, 'content', 'GPT_results')
  model_prefix = 'GPT'

def separateJustification(LLMoutString, criterion):
  """
  The Justifications returned by the models often break the json format, so I extract them
  """
  search_expression = '("'+criterion+'":[^\{]+\{[^\}]*"Justification":)([^\}]+)("Score":[^\}]+\})'
  if re.search(search_expression, LLMoutString):
    justificationRemoved = re.sub(search_expression, '\g<1> "", \g<3>',  LLMoutString)
    justification = re.sub('^.*'+search_expression+'.*$', '\g<2>',  LLMoutString)
  else:
    justificationRemoved = LLMoutString
    justification = ''
  return justificationRemoved, justification

def loadDataPoint(dbfile_x, model):
  eval_missing = None
  wrong_score = None
  dico_key = 'scores_'+str(model)
  formatted_scores = {}
  # load data with pickle
  dp = pickle.load(dbfile_x)
  if dico_key in dp:
    print(dp['id'])
    print(dp['triples'])
    print(dp['text'])
    justifications = []
    # pickle.load uses single quotes, whereas json.load expects double quotes
    # Gemini adds a node "query" in the json, unlike OpenAI's models
    LLMout_string = str(dp[dico_key]).replace("'query'", '"query"')
    LLMout_string = LLMout_string.replace("```json", "")
    LLMout_string = LLMout_string.replace("```", "")
    LLMout_string = LLMout_string.replace("'No-Omissions'", '"No-Omissions"')
    # There's a typo in one of the Gemini outputs
    LLMout_string = LLMout_string.replace("'No-Omissons'", '"No-Omissions"')
    LLMout_string = LLMout_string.replace('"No-Omissons"', '"No-Omissions"')
    LLMout_string = LLMout_string.replace("'No-Additions'", '"No-Additions"')
    LLMout_string = LLMout_string.replace("'Grammaticality'", '"Grammaticality"')
    LLMout_string = LLMout_string.replace("'Fluency'", '"Fluency"')
    # Sometimes justifications are followed by single quotes, sometimes by double quotes
    LLMout_string = LLMout_string.replace("'Justification': '", '"Justification": "').replace("'Justification'", '"Justification"')
    LLMout_string = LLMout_string.replace("', 'Score'", '", "Score"').replace("'Score'", '"Score"')
    if LLMout_string == 'None':
      eval_missing = dp['id']
    else:
      # print(LLMout_string)
      # LLMout_string = re.sub('("No-Omissions":[^\{]+\{"Justification":)([^\}]+)("Score":[^\}]+\})', '\g<1> "", \g<3>',  LLMout_string)
      LLMout_string, justifNoOm = separateJustification(LLMout_string, 'No-Omissions')
      justifications.append(justifNoOm)
      LLMout_string, justifNoAdd = separateJustification(LLMout_string, 'No-Additions')
      justifications.append(justifNoAdd)
      LLMout_string, justifGram = separateJustification(LLMout_string, 'Grammaticality')
      justifications.append(justifGram)
      LLMout_string, justifFlu = separateJustification(LLMout_string, 'Fluency')
      justifications.append(justifFlu)
      print(justifications)
      LLMout_string = LLMout_string.replace("'1'", '"1"')
      LLMout_string = LLMout_string.replace("'2'", '"2"')
      LLMout_string = LLMout_string.replace("'3'", '"3"')
      LLMout_string = LLMout_string.replace("'4'", '"4"')
      LLMout_string = LLMout_string.replace("'5'", '"5"')
      LLMout_string = LLMout_string.replace("'6'", '"6"')
      LLMout_string = LLMout_string.replace("'7'", '"7"')
      scores_json = json.loads(LLMout_string)
      clean_scores_json = None
      # Gemini adds a node "query" in the json, unlike OpenAI's models
      if 'query' in scores_json:
        clean_scores_json = scores_json['query']
      else:
        clean_scores_json = scores_json

      gram_score = int(clean_scores_json['Grammaticality']['Score'])
      flu_score = int(clean_scores_json['Fluency']['Score'])
      no_om_score = int(clean_scores_json['No-Omissions']['Score'])
      no_ad_score = int(clean_scores_json['No-Additions']['Score'])

      if (gram_score not in model_scores) or (flu_score not in model_scores) or (no_om_score not in model_scores) or (no_ad_score not in model_scores):
        wrong_score = dp['id']

      formatted_scores["eid"] = dp['id']
      formatted_scores["annotator_id"] = str(zip_model)
      formatted_scores["no-omissions"] = no_om_score
      formatted_scores["no-additions"] = no_ad_score
      formatted_scores["grammaticality"] = gram_score
      formatted_scores["fluency"] = flu_score

      print(f"Gram: {gram_score}; Flu: {flu_score}; NoOm: {no_om_score}; NoAd: {no_ad_score}.")
      print('')

  return formatted_scores, eval_missing, wrong_score

print(path_dir_unzipped)
print(model_prefix)
eval_files = glob.glob(os.path.join(path_dir_unzipped, '*'))
evals_missing = []
wrong_scores = []
all_scores = []
for filepath in eval_files:
  print(filepath)
  dbfile_x = open(filepath, 'rb')
  formatted_scores, eval_missing, wrong_score = loadDataPoint(dbfile_x, model_prefix)
  if eval_missing != None:
    evals_missing.append(eval_missing)
  if wrong_score != None:
    wrong_scores.append(wrong_score)
  dbfile_x.close()
  all_scores.append(formatted_scores)
print(f'Missing evaluations: {evals_missing}')
print(f'Wrong scores: {wrong_scores}')

# Save all scores into a json file
path_json_out = zip_language+'_'+zip_model+'_scores.json'
with codecs.open(path_json_out, 'w', 'utf-8') as outfile:
  json.dump(all_scores, outfile)